<font size="6"><b>Lecture 02: GEE Image Manipulation (exercises)</b></font>

# Initialize

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='your-project-id')  # <--- replace with your project ID

In [ ]:
import geemap

# EX1: calculation of spectral indices (band arithmetic)

## 1.1. select image

**Existing Sentinel-2 collections and use case**

| GEE Collection | Data Type / Processing Level |
|----------------|-----------------------------|
| [COPERNICUS/S2_HARMONIZED](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED) | **Top-of-Atmosphere Reflectance (TOA)** — corrected for solar geometry but still includes atmospheric effects |
| [COPERNICUS/S2_SR_HARMONIZED](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) | **Surface Reflectance (SR)** — atmospherically corrected (Sen2Cor) |

**Glossary**

- HARMONIZED:  
    ESA introduced a **band-dependent offset** to reflectance bands after 24 Jan 2022. The *HARMONIZED* collections **remove this offset**, ensuring consistency across the full archive. Use **HARMONIZED** unless you need legacy data behavior.


In [ ]:
# Define ROI point (EX: Cancún)
roi = ee.Geometry.Point(-86.85, 21.17)

# Import and filter imagery by location / date / cloud
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(roi)
            .filterDate('2020-01-01', '2024-10-01')
            .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 10))
            .sort("CLOUD_COVERAGE_ASSESSMENT")
            .first()
            )

# Map Natural color image
Map = geemap.Map()
Map.addLayerControl()
Map.centerObject(roi, 10)
Map.addLayer(image, {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2000
}, 'Natural color')
Map.addLayer(roi, {'color':'red'})
# Map.add_basemap('OpenTopoMap')
Map

In [ ]:
image

## 1.2. calculate spectral indices

### calculate NDVI (using math expression)

$NDVI = \frac{NIR - Red}{NIR + Red}$

Sentinel-2 (Surface Reflectance, HARMONIZED) bands (
<a href="https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands">COPERNICUS_S2_SR_HARMONIZED</a>):
<br>

<table class="eecat">
  <tbody>
    <tr>
      <th scope="col">Name</th>
      <th scope="col">Pixel Size</th>
      <th scope="col">Wavelength</th>
      <th scope="col">Description</th>
    </tr>
    <tr><td><code>B1</code></td><td>60 m</td><td>0.443 μm (S2A)</td><td>Aerosols</td></tr>
    <tr><td><code>B2</code></td><td>10 m</td><td>0.496 μm (S2A)</td><td>Blue</td></tr>
    <tr><td><code>B3</code></td><td>10 m</td><td>0.560 μm (S2A)</td><td>Green</td></tr>
    <tr><td><code>B4</code></td><td>10 m</td><td>0.664 μm (S2A)</td><td>Red</td></tr>
    <tr><td><code>B5</code></td><td>20 m</td><td>0.703 μm (S2A)</td><td>Red Edge 1</td></tr>
    <tr><td><code>B6</code></td><td>20 m</td><td>0.740 μm (S2A)</td><td>Red Edge 2</td></tr>
    <tr><td><code>B7</code></td><td>20 m</td><td>0.782 μm (S2A)</td><td>Red Edge 3</td></tr>
    <tr><td><code>B8</code></td><td>10 m</td><td>0.835 μm (S2A)</td><td>Near Infrared (NIR)</td></tr>
    <tr><td><code>B8A</code></td><td>20 m</td><td>0.864 μm (S2A)</td><td>Red Edge 4</td></tr>
    <tr><td><code>B9</code></td><td>60 m</td><td>0.955 μm (S2A)</td><td>Water Vapor</td></tr>
    <tr><td><code>B11</code></td><td>20 m</td><td>1.613 μm (S2A)</td><td>Shortwave Infrared (SWIR 1)</td></tr>
    <tr><td><code>B12</code></td><td>20 m</td><td>2.202 μm (S2A)</td><td>Shortwave Infrared (SWIR 2)</td></tr>
  </tbody>
</table>

**NDVI Interpretation Table**

| NDVI Range | Vegetation Condition        | Interpretation                            |
|------------|-----------------------------|-------------------------------------------|
| -1.0 to -0.2 | Water / Snow / Clouds       | Strong negative reflectance (water bodies, snow, shadows) |
| -0.2 to 0.0  | Barren Land / Rock / Sand   | No vegetation present                     |
| 0.0 to 0.2   | Sparse Vegetation / Urban   | Built-up areas or dry grassland           |
| 0.2 to 0.5   | Moderate Vegetation         | Shrubland, crops, mixed vegetation        |
| 0.5 to 0.7   | Dense Vegetation            | Healthy crops, temperate forests          |
| 0.7 to 1.0   | Very Dense / Tropical Canopy | Rainforests, peak photosynthetic activity |

Reference: [ee.Image.expression](https://developers.google.com/earth-engine/apidocs/ee-image-expression)

In [ ]:
# Calculate NDVI index (using math expression)
ndvi = image.expression(
    # write your code here
    )

# Add NDVI index on map
palette = ['red', 'white', 'green']
Map.addLayer(ndvi, {
    'min': -1,
    'max': 1,
    'palette': palette
}, 'NDVI manual')

# Add colorbar on map
vis_params = {'min': -1, 'max': 1, 'palette': palette}
cbar = Map.add_colorbar(vis_params, label="NDVI")

Map

### calculate NDVI (using adhoc method)

Reference: [ee.Image.normalizedDifference](https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference)

In [ ]:
# Calculate NDVI using normalizedDifference method
NDVI = image.normalizedDifference(['B8', 'B4'])

palette = ['red', 'white', 'green']
Map.addLayer(NDVI, {
    'min': -1,
    'max': 1,
    'palette': palette
}, 'NDVI')

Map

### calculate EVI (using math expression)

<u>**Note**</u>:

In Google Earth Engine, the collections store values as **scaled integers** to save storage. This means that the values need to be multiplied by a **scale factor** to convert them to values with physical meaning. In the case of Sentinel-2 Surface Reflectance products, to obtain physically meaningful reflectance values, the scale factor must be applied (typically **0.0001**).

If $N'$ and $R'$ are the **stored integer values** (scaled), and $s$ is the scale factor, then the **true reflectances** are:

$$
N = N' \cdot s \quad\text{and}\quad R = R' \cdot s
$$

Therefore, for pure ratio indices like NDVI, the scale factor **cancels out**, making it unnecessary to apply the scale factor before computing the index:

$$
\text{NDVI}' = \frac{N' - R'}{N' + R'}
= \frac{sN - sR}{sN + sR} = \frac{s(N - R)}{s(N + R)} = \frac{N - R}{N + R}
$$

However, for indices like EVI that involve additional constants, the scale factor does not cancel out, and it is necessary to apply the scale factor to the bands before computing the index.

In [ ]:
# Compute the EVI using math expression
evi = image.expression(
    # write your code here
    # Recall to multiply bands by the scale factor 0.0001
    )

Map2 = geemap.Map()
Map2.centerObject(image, 9)
Map2.addLayer(evi, {'min': 0, 'max': 1, 'palette': ['red', 'white', 'green']}, 'EVI')
Map2

## 1.3. thresholding image

### binary thresholding (.gt, .lt)

References:
- [ee.Image.gt](https://developers.google.com/earth-engine/apidocs/ee-image-gt)
- [ee.Image.lt](https://developers.google.com/earth-engine/apidocs/ee-image-lt)

In [ ]:
# Implement a binary threshold
threshold = 0.5
img_thresh_bin = ndvi.gt(threshold)

# Map thresholded image (2 classes)
palette = ['white', 'green']
Map.addLayer(img_thresh_bin, {
    'min': 0,
    'max': 1,
    'palette': palette
}, 'Non-forest vs. Forest')

# Add colorbar on map
vis_params = {'min': 0, 'max': 1, 'palette': palette}

Map.centerObject(roi, 10)
Map.add_colorbar(vis_params, label="NDVI thresholded (2-classes)")

Map

### advanced thresholding (.where)

In [ ]:
threshold_1 = -0.1  # set water threshold
threshold_2 = 0.5   # set vegetation threshold

img_thresh = ee.Image(1)  # Initialize new thresholded image with all values = 1
img_thresh = img_thresh.clip(ndvi.geometry())   # Use clip to constrain size of the ndvi image
img_thresh = img_thresh.where(ndvi.lte(threshold_1), 0)  # Make all NDVI values <= threshold_1 equal 0
img_thresh = img_thresh.where(ndvi.gte(threshold_2), 2) # Make all NDVI values >= threshold_2 equal 2

# Map thresholded image (3 classes)
palette = ['blue', 'white', 'green']
Map.addLayer(img_thresh, {
    'min': 0,
    'max': 2,
    'palette': palette
}, 'Water, Non-forest, Forest')

# Add colorbar on map
vis_params = {'min': 0, 'max': 1, 'palette': palette}
Map.add_colorbar(vis_params, label="NDVI thresholded (3-classes)")

Map

## 1.4. masking image

In [ ]:
# NDVI masking in GEE

mask = img_thresh_bin.eq(1)                     # Create a binary mask of non-forest
img_masked = img_thresh_bin.updateMask(mask)    # Update the img_thresh_bin mask with the non-forest mask
mask_final = img_masked.mask()                  # Updated mask

product_mask = image.mask()                  # Updated mask

# Visualize masked image
Map.addLayer(img_masked, {
    'min': 0,
    'max': 1,
    'palette': ['green']
    }, 'Masked Forest Layer')

# Visualize updated mask
Map.addLayer(mask_final, {}, 'Mask')

# Map.addLayer(product_mask, {}, 'Product Mask')

Map

## 1.5. (remapping values in image)

In [ ]:
# Remap the values from the thresholded image
img_thresh_remap = img_thresh.remap(
    [0, 1, 2],    # Existing values
    [9, 11, 10]   # Remapped values
    )

# Visualize the remapped thresholded image
vis_params = {'min': 9, 'max': 11, 'palette': ['blue', 'green', 'white']}
Map.addLayer(img_thresh_remap, vis_params, 'Remapped Values')

# Add colorbar on map
cbar = Map.add_colorbar(vis_params, label="mask (remapped values)")

Map


## 1.6. analyze image

### calculate vegetation area

In [ ]:
# Create a pixel area image
img_pixArea = ee.Image.pixelArea()

# Apply the vegetation mask to the pixel area image
mask_area = img_pixArea.updateMask(mask)

Map = geemap.Map()
Map.addLayerControl()
Map.centerObject(roi, 13)
Map.addLayer(img_masked, {
    'min': 0,
    'max': 1,
    'palette': ['green']
    }, 'Masked Forest Layer')
Map.addLayer(mask_area, {}, 'mask area')
Map

In [ ]:
# Sum the area of vegetation pixels
crs = mask.projection()  # coordinate reference system
crsTransform = mask.projection().getInfo()['transform']
geometry = mask.geometry()  # sum area over entire image

area = mask_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geometry,
    crs=crs,
    crsTransform=crsTransform,
    maxPixels=1e10,
)

In [ ]:
# Fetch the summed area property from the resulting dictionary and convert to square meters to square kilometers.
square_meters = area.getNumber('area').round()
square_kilometers = square_meters.divide(1e6).round()

print('Vegetation area = {} m2'.format(square_meters.getInfo()))
print('Vegetation area = {} km2'.format(square_kilometers.getInfo()))

### export numpy array

In [ ]:
# Plot map to draw region to extract
Map

In [ ]:
mypolygon = Map.draw_last_feature
region = mypolygon.geometry()

In [ ]:
arr_rgb = geemap.ee_to_numpy(image,
                             region=region,
                             bands=['B4', 'B3', 'B2'],
                             scale=10)


In [ ]:
Map.addLayer(region, {'color': 'black', 'opacity':.5}, 'region to export')
Map

In [ ]:
import matplotlib.pyplot as plt

# Normalize rgb values (need to have array in range 0-1 for dtype float in order to plot)
arr_rgb_n = arr_rgb/arr_rgb.max()

# Plot
plt.figure()
plt.imshow(arr_rgb_n)

plt.figure()
plt.imshow(arr_rgb, cmap='gray')


In [ ]:
print(arr_rgb.min(), arr_rgb.max())
print(arr_rgb_n.min(), arr_rgb_n.max())

# EX2: Image composing (remove/replace clouds)
Refs:
- https://courses.geemap.org/gee_intro/Image/conditional_operations/#expressions
- Cardille et al. 2018 chap.15 p.279

In [ ]:
# Load a cloudy Landsat 8 image
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130603')

# Load another image to replace cloudy pixels
replacement = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130416')

# Compute a cloud score band
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')

# Set cloudy pixels to the other image.
replaced = image.where(cloud.gt(10), replacement)

# Display result
Map = geemap.Map()
Map.addLayerControl()
Map.centerObject(image, 9)
Map.addLayer(
    image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'original image (cloudy)'
)
Map.addLayer(cloud, {}, 'cloud score')
Map.addLayer(
    replaced, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'fused image (clouds replaced)'
)
Map